In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [34]:
ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

In [35]:
ratings

<MapDataset element_spec={'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(), dtype=tf.float32, name=None)}>

In [38]:
from pymongo import MongoClient
import pandas as pd

mongo = MongoClient()
data = list(mongo["data_1"]["events"].find({},{"_id": 0}))
df = pd.DataFrame(data)

BEHAVIORS = {
    "apply": 5.0,
    "search": 4.0,
    "like": 4.0,
    "view": 3.0,
    "ignore": 1.0,
    "dislike": 0.0,
    "hated": 0.0,
}

def create_rating(behavior):
    return BEHAVIORS.get(behavior, 0)
    
df["user_rating"] = df.behavior.apply(create_rating)
ratings = df[["userid", "jobId", "user_rating"]]

tensor_slices = {
    "user_id": ratings.userid.values.tolist(),
    "movie_title": ratings.jobId.values.tolist(),
    "user_rating": ratings.user_rating.values.tolist()
}

# movies = tf.data.Dataset.from_tensor_slices(np.unique(ratings['jobId'].to_numpy()))
ratings = tf.data.Dataset.from_tensor_slices(tensor_slices)

In [39]:
ratings

<TensorSliceDataset element_spec={'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(), dtype=tf.float32, name=None)}>

In [40]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [41]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [42]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [43]:
RankingModel()((["User_Data_Other_Senior_Test9"], ["Job_Fresher_Backend_Developer_DN_Fresher_6CFZ4M"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.01600142]], dtype=float32)>

In [44]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [45]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [46]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [47]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [48]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 1s 52ms/step - root_mean_squared_error: 1.6806 - loss: 2.6682 - regularization_loss: 0.0000e+00 - total_loss: 2.6682
Epoch 2/3
10/10 [==============================] - 0s 4ms/step - root_mean_squared_error: 1.3348 - loss: 1.7799 - regularization_loss: 0.0000e+00 - total_loss: 1.7799
Epoch 3/3
10/10 [==============================] - 0s 4ms/step - root_mean_squared_error: 1.3240 - loss: 1.7454 - regularization_loss: 0.0000e+00 - total_loss: 1.7454


In [49]:
model.evaluate(cached_test, return_dict=True)

/home/spark/miniconda3/envs/recommend/lib/python3.9/site-packages/keras/utils/generic_utils.py:239: RuntimeWarning: divide by zero encountered in log10
  numdigits = int(np.log10(self.target)) + 1


OverflowError: cannot convert float infinity to integer

In [51]:
test_ratings = {}
test_movie_titles = ["Job_Middle_Data_Analyst_HCM_Middle_ZVRYRB", 
                     "Job_Senior_Data_Scientist_other_Senior_TWBT6U", 
                     "Job_Middle_Data_Analyst_HCM_Middle_15YON8",
                     "Job_Junior_Data_Scientist_HN_Junior_VHTUHL"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["User_Senior_Data_HCM_12"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
Job_Middle_Data_Analyst_HCM_Middle_ZVRYRB: [[2.3891296]]
Job_Senior_Data_Scientist_other_Senior_TWBT6U: [[2.3891296]]
Job_Middle_Data_Analyst_HCM_Middle_15YON8: [[2.3891296]]
Job_Junior_Data_Scientist_HN_Junior_VHTUHL: [[2.3891296]]


In [15]:
tf.saved_model.save(model, "export")

INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 32]), tf.float32, tf.int64, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f90d0056a30>, 140259943511504), {}).


INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 32]), tf.float32, tf.int64, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f90d0056a30>, 140259943511504), {}).


INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 32]), tf.float32, tf.int64, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f90d00744f0>, 140259943292400), {}).


INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 32]), tf.float32, tf.int64, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f90d00744f0>, 140259943292400), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64, 256), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f90d007e8b0>, 140259943291040), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64, 1), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f90d00c0910>, 140259943186064), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f90d0062f10>, 140259943186304), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f90d0062f10>, 140259943186304), {}).


INFO:tensorflow:Assets written to: export/assets


INFO:tensorflow:Assets written to: export/assets


In [52]:
loaded = tf.saved_model.load("export")

loaded({"user_id": np.array(["User_Senior_Data_HCM_12"]), "movie_title": ["Job_Middle_Data_Analyst_HCM_Middle_ZVRYRB"]}).numpy()

array([[3.5470235]], dtype=float32)